# Ripple Carry Adder Workbok
**What is this workbook?** 
A workbook is a collection of problems, accompanied by solutions to them. The explanations focus on the logical steps required to solve a problem; they illustrate the concepts that need to be applied to come up with a solution to the problem, explaining the mathematical steps required.

Note that a workbook should not be the primary source of knowledge on the subject matter; it assumes that you've already read a tutorial or a textbook and that you are now seeking to improve your problem-solving skills. You should attempt solving the tasks of the respective kata first, and turn to the workbook only if stuck or for reinforcement. While a textbook emphasizes knowledge acquisition, a workbook emphasizes skill acquisition.

This workbook describes the solutions to the problems offered in the [Ripple Carry Adder Kata](./RippleCarryAdder.ipynb). 
Since the tasks are offered as programming problems, the explanations also cover some elements of Q# that might be non-obvious for a novice.

**What you should know for this workbook**

You should be familiar with the following concepts and associated techniques **prior to** beginning work on the Basic Gates Quantum Kata.

1. [Basic Gates Kata](../BasicGates/BasicGates.ipynb). 
2. [Single-qubit gates](../tutorials/SingleQubitGates/SingleQubitGates.ipynb).
3. List of basic gates available in Q# can be found at [Microsoft.Quantum.Intrinsic](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic).
4. [Syntax of flow control statements in Q#](https://docs.microsoft.com/azure/quantum/user-guide/language/statements/iterations) 
5. [Q# conditional branching](https://docs.microsoft.com/azure/quantum/user-guide/language/statements/conditionalbranching) documentation.

You can also consult the [complete Quantum Katas learning path](https://github.com/microsoft/QuantumKatas#learning-path).

## Part I. Simple Adder Outputting to Empty Qubits


### Theory

* [Classical binary adder on Wikipedia](https://en.wikipedia.org/wiki/Adder_(electronics)).
* Part 2 of the [paper on quantum binary addition](https://arxiv.org/pdf/quant-ph/0008033.pdf) by Thomas G. Draper explains how to adapt the classical adder to a quantum environment.

### Task 1.1. Summation of two bits

**Inputs:**

  1. qubit `a` in an arbitrary state $|\phi\rangle$,  
  2. qubit `b` in an arbitrary state $|\psi\rangle$,  
  3. qubit `sum` in state $|0\rangle$.

**Goal:** Transform the `sum` qubit into the lowest bit of the binary sum of $\phi$ and $\psi$.

* $|0\rangle + |0\rangle \to |0\rangle$
* $|0\rangle + |1\rangle \to |1\rangle$
* $|1\rangle + |0\rangle \to |1\rangle$
* $|1\rangle + |1\rangle \to |0\rangle$

Any superposition should map appropriately. 

**Example:** (Recall that $|+\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$, $|-\rangle = \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$)

$|+\rangle \otimes |-\rangle \otimes |0\rangle \to \frac{1}{2}(|000\rangle + |101\rangle - |011\rangle - |110\rangle)$

### Solution

> In this workbook we will reason about the computations acting on basis states, as opposed to arbitrary superposition states. 
> (Since all computations will be implemented using quantum gates, they'll be linear, and behave correctly on superpositions if they are correct on all basis states.) 
> This will allow us to use qubit states and the classical values stored in them interchangeably.

Clearly, the expression we're looking for is $a \oplus b$ - sum of bits $a$ and $b$ modulo 2.

If we apply $CNOT(a,sum)$, then $a=a$, $b=b$, and $sum$ becomes $0 \oplus a = a$.

Again applying $CNOT(b,sum)$, we get $a=a$, $b=b$, and $sum$ becomes $sum \oplus b = a \oplus b$. That's exactly what we're looking for!

In [ ]:
%kata T11_LowestBitSum

operation LowestBitSum (a : Qubit, b : Qubit, sum : Qubit) : Unit is Adj {
    CNOT(a, sum);
    CNOT(b, sum);
}

### Alternate Solution

If we apply $CNOT(a,b)$ , then $a = a$, $b = a \oplus b$, and $sum = 0$.

Again applying $CNOT(b,sum)$, we get $a = a$, $b = a \oplus b$, and $sum = sum \oplus b = a \oplus b$.

Now by applying $CNOT(a,b)$, we get $a = a$, $b = a \oplus (a \oplus b) = b$, $sum = a \oplus b$, and thus we will restore the previous value of $b$.

In [ ]:
%kata T11_LowestBitSum

operation LowestBitSum (a : Qubit, b : Qubit, sum : Qubit) : Unit is Adj {
    CNOT(a, b);
    CNOT(b, sum);
    CNOT(a, b);
}

[Return to Task 1.1 of the Ripple Carry Adder kata.](./RippleCarryAdder.ipynb#Task-1.1.-Summation-of-two-bits)

### Task 1.2. Carry of two bits

**Inputs:**

  1. qubit `a` in an arbitrary state $|\phi\rangle$,
  2. qubit `b` in an arbitrary state $|\psi\rangle$,
  3. qubit `carry` in state $|0\rangle$.

**Goal:** Set the `carry` qubit to $|1\rangle$ if the binary sum of $\phi$ and $\psi$ produces a carry.

* $|0\rangle$ and $|0\rangle \to |0\rangle$
* $|0\rangle$ and $|1\rangle \to |0\rangle$
* $|1\rangle$ and $|0\rangle \to |0\rangle$
* $|1\rangle$ and $|1\rangle \to |1\rangle$

Any superposition should map appropriately. 

**Example:**

$|+\rangle \otimes |-\rangle \otimes |0\rangle \to \frac{1}{2}(|000\rangle + |100\rangle - |010\rangle - |111\rangle)$

### Solution
Clearly, $carry = a \text{ AND } b$.

If we apply $CCNOT(a, b, carry)$, then $a=a$, $b=b$, and $carry$ becomes $(a \text{ AND } b) \oplus carry = (a \text{ AND } b) \oplus 0 = (a \text{ AND } b)$.

In [ ]:
%kata T12_LowestBitCarry

operation LowestBitCarry (a : Qubit, b : Qubit, carry : Qubit) : Unit is Adj {
    CCNOT(a, b, carry);
}

[Return to Task 1.2 of the Ripple Carry Adder kata.](./RippleCarryAdder.ipynb#Task-1.2.-Carry-of-two-bits)

### Task 1.3. One-bit adder

**Inputs:**

  1. qubit `a` in an arbitrary state $|\phi\rangle$,
  2. qubit `b` in an arbitrary state $|\psi\rangle$,
  3. two qubits `sum` and `carry` in state $|0\rangle$.

**Goals:**

* Transform the `sum` qubit into the lowest bit of the binary sum of $\phi$ and $\psi$.
* Transform the `carry` qubit into the carry bit produced by that sum.

### Solution
This task is just a combination of the two previous ones. Using [task 1.1](./Workbook_RippleCarryAdder.ipynb#Task-1.1.-Summation-of-two-bits) and [task 1.2](./Workbook_RippleCarryAdder.ipynb#Task-1.2.-Carry-of-two-bits), 
we can get the solution below.

In [ ]:
%kata T13_OneBitAdder

operation OneBitAdder (a : Qubit, b : Qubit, sum : Qubit, carry : Qubit) : Unit is Adj {
    LowestBitSum(a, b, sum);
    LowestBitCarry(a, b, carry);
}

[Return to Task 1.3 of the Ripple Carry Adder kata.](./RippleCarryAdder.ipynb#Task-1.3.-One-bit-adder)

More solutions comming soon..